In [1]:
from dataclasses import dataclass
from datetime import datetime
from uuid import uuid4

@dataclass
class User:
    name: str
    id: str = None

    # auto generate user id if one wasn't provided
    def __post_init__(self):
        if self.id is None:
            self.id = str(uuid4())

@dataclass
class OrderItem:
    name: str
    cost: float
    user: User

@dataclass
class Order:
    orderTime: datetime
    payer: User
    items: list[OrderItem]

    @property
    def order_cost(self):
        return sum([i.cost for i in self.items])


In [2]:
bob = User('Bob')
jeremy = User('Jeremy')

orders = [
    Order(
        datetime.now(),
        bob,
        [
            OrderItem('cappuccino', 3.5, bob),
            OrderItem('coffee', 2.5, jeremy)
        ]
    ),
    Order(
        datetime.now(),
        jeremy,
        [
            OrderItem('cappuccino', 1.5, bob),
            OrderItem('coffee', 2.5, jeremy)
        ]
    )
]

In [6]:
from collections import defaultdict

### Simple function that returns who pays next by finding who owes the most to the group
###  a.k.a. who has the highest debt
def get_next_payer(orders: list[Order]):
    member_debt = defaultdict(int)

    # Loop over orders to calculate member debt
    for order in orders:
        # process credit to payer of order
        member_debt[order.payer.id] -= order.order_cost

        # process debt for others on order
        for item in order.items:
            member_debt[item.user.id] += item.cost
    
    # now select the member with the highest debt to pay next
    return sorted(member_debt.items(), key=lambda i: i[1],  reverse=True)[0][0]

In [7]:
print(bob)
print(jeremy)

User(name='Bob', id='2569bd53-c663-431f-9730-7446ffe9182a')
User(name='Jeremy', id='5815596c-2d1b-4bf4-837b-51944a48abfe')


In [8]:
get_next_payer(orders)

'5815596c-2d1b-4bf4-837b-51944a48abfe'

- Bob paid for the first order, so Jeremy owes \$2.5
- Jermey paid for the second order.  Bob's item only cost \$1.5, so Jeremey still owes \$1 (\$2.5 - \$1.5)
- In both orders the payer pays for their own item.

We should expect that Jeremy still owes \$1 and should pay next since Bob owes \$0 debt.

`get_next_payer` returns Jeremy's id, which is correct since he owes the most debt
